In [1]:
ls

DI_buildout.ipynb      drug_info_raw.csv      drugs_labelled.csv
auc_exp1.2-tki.R       drugbank_scrape.ipynb  tki_labels.txt


In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("drug_info_raw.csv")

In [3]:
data

,NAME,Summary,DrugBank Accession Number,Background,Groups,Weight,Indication
0,Afatinib 1/2,Afatinib is an antineoplastic agent used for t...,DB08916,Afatinib is a 4-anilinoquinazoline tyrosine ki...,Approved,485.938,Afatinib is a kinase inhibitor indicated as mo...
1,Afatinib 2/2,Afatinib is an antineoplastic agent used for t...,DB08916,Afatinib is a 4-anilinoquinazoline tyrosine ki...,Approved,485.938,Afatinib is a kinase inhibitor indicated as mo...
2,AKT inhibitor VIII 1/2,NaN,NaN,NaN,NaN,NaN,NaN
3,AKT inhibitor VIII 2/2,NaN,NaN,NaN,NaN,NaN,NaN
4,Avagacestat 1/2,NaN,DB11893,Avagacestat has been investigated for the basi...,Investigational,520.880,Not Available
...,...,...,...,...,...,...,...
261,Z-LLNle-CHO,NaN,NaN,NaN,NaN,NaN,NaN
262,ZG-10,NaN,NaN,NaN,NaN,NaN,NaN
263,Zibotentan,NaN,DB06629,Not Available,Investigational,424.440,Investigated for use/treatment in prostate can...
264,ZM447439,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
pd.options.display.max_colwidth = 1000
data["Indication"]

0      Afatinib is a kinase inhibitor indicated as monotherapy 3 for the first-line Label treatment of (a) Epidermal Growth Factor Receptor (EGFR) TKI (tyrosine kinase inhibitor)-naive adult patients with locally advanced or metastatic non-small cell lung cancer (NSCLC) whose tumours have non-resistant EGFR mutations as detected by an FDA-approved test Label, and (b) adult patients with locally advanced or metastatic NSCLC of squamous histology progressing on or after platinum-based chemotherapy Label,3.\nRecently, as of January 2018, the US FDA approved a supplemental New Drug Application for Boehringer Ingelheim's Gilotrif (afatinib) for the first line treatment of patients with metastatic non-small cell lung cancer (NSCLC) whose tumors have non-resistant epidermal growth factor receptor (EGFR) mutations as detected by an FDA-approved test 4. The new label includes data on three additional EGFR mutations: L861Q, G719X and S768I 4.\nReduce drug development failure ratesBuild, train, &

In [5]:
#lets do a really naive thing (names from R)
cancers = ["LUNG", "LARGE_INTESTINE", "HAEMATOPOIETIC_AND_LYMPHOID_TISSUE", "URINARY_TRACT", "BONE", "SKIN", 
"BREAST","LIVER","PLEURA", "OVARY","OESOPHAGUS","ENDOMETRIUM", "CENTRAL_NERVOUS_SYSTEM","SOFT_TISSUE","PANCREAS",                          
"FIBROBLAST","STOMACH","KIDNEY", "UPPER_AERODIGESTIVE_TRACT","AUTONOMIC_GANGLIA","BILIARY_TRACT",                
"THYROID","SALIVARY_GLAND","SMALL_INTESTINE", "PROSTATE","CERVIX","ADRENAL_CORTEX", "PLACENTA"]         


cancers = list(map(lambda s: s.lower(), cancers))

In [6]:
#STEP 1: go through background, indication, and summary cols to see if keyword in text
nonempty = []

for i in range(data.shape[0]):
    drug_tags = []
    empty = True
    for c in cancers:
        summary = data.iloc[i]["Summary"]
        background = data.iloc[i]["Background"]
        indication = data.iloc[i]["Indication"]
        
        if type(summary)!=float:
            empty = False             
        
        if type(background)!=float:
            empty = False
                
        if type(indication)!=float:
            empty = False
    
    if not empty:
        nonempty.append(data.iloc[i]["NAME"])


In [7]:
#STEP 2: go through all the nonempty ones to see if we can add more words 
#from online 
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nathanielnethercott/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nathanielnethercott/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
#build out some search terms -- THESE ARE NOT REALLY RELEVANT MOST OF THE TIME 
from nltk.corpus import wordnet

#new querries
search_terms = {}

for c in cancers:
    synonyms = []
    
    #get rid of underscore
    word = c
    if(len(c.split("_"))>1):
        word = " ".join(c.split("_"))
        
    synonyms.append(word)
        
    for syn in wordnet.synsets(word):
        for lm in syn.lemmas():
            lm = lm.name()
            if(len(lm.split("_"))>1):
                lm = " ".join(lm.split("_"))
                
            synonyms.append(lm) #adding into synonyms
    search_terms[c] = list(set(synonyms))

search_terms.keys()

dict_keys(['lung', 'large_intestine', 'haematopoietic_and_lymphoid_tissue', 'urinary_tract', 'bone', 'skin', 'breast', 'liver', 'pleura', 'ovary', 'oesophagus', 'endometrium', 'central_nervous_system', 'soft_tissue', 'pancreas', 'fibroblast', 'stomach', 'kidney', 'upper_aerodigestive_tract', 'autonomic_ganglia', 'biliary_tract', 'thyroid', 'salivary_gland', 'small_intestine', 'prostate', 'cervix', 'adrenal_cortex', 'placenta'])

In [9]:
#manually add some other words I know about 
search_terms['lung'].extend(['pulmonary', 'lungs'])
search_terms['kidney'].extend(['kidneys', 'renal'])
search_terms['pancreas'].extend(['pancreatic'])
search_terms['ovary'].extend(['ovaries', 'ovarian'])
search_terms['liver'].extend(['hepatic'])
search_terms['haematopoietic_and_lymphoid_tissue'].extend(['haematopoietic', 'lymphoid'])
search_terms['upper_aerodigestive_tract'].extend(['aerodigestive'])
search_terms['adrenal_cortex'].extend(['adrenal'])
search_terms['urinary_tract'].extend(['urinary'])
search_terms['central_nervous_system'].extend(['nervous system', 'glioblastoma', 'neurofibromatosis', 'anaplastic astrocytoma'])


#search_terms

In [10]:
#manual stuff to make it make sense 
search_terms['bone'] = ['os', 'bone', 'osseous tissue', 'marrow', 'leukemia', 'myelogenous', 'myeloma', 'myeloid']
search_terms['skin'] = ['skin', 'cutis', 'melanoma', 'basal carcinoma', 'basal cell carcinoma','cutaneous']
search_terms['breast'] = ['breast', 'bosom']
search_terms['liver'].remove('liver-colored')
search_terms['stomach'] = ['stomach', 'abdomen', 'gastric', 'gastrointestinal']
search_terms['cervix'].remove('neck')
search_terms['oesophagus'].remove('gullet')
search_terms['oesophagus'].remove('gorge')

search_terms

{'lung': ['lung', 'pulmonary', 'lungs'],
 'large_intestine': ['large intestine'],
 'haematopoietic_and_lymphoid_tissue': ['haematopoietic and lymphoid tissue',
  'haematopoietic',
  'lymphoid'],
 'urinary_tract': ['urinary tract', 'urinary'],
 'bone': ['os',
  'bone',
  'osseous tissue',
  'marrow',
  'leukemia',
  'myelogenous',
  'myeloma',
  'myeloid'],
 'skin': ['skin',
  'cutis',
  'melanoma',
  'basal carcinoma',
  'basal cell carcinoma',
  'cutaneous'],
 'breast': ['breast', 'bosom'],
 'liver': ['liver', 'hepatic'],
 'pleura': ['pleura'],
 'ovary': ['ovary', 'ovaries', 'ovarian'],
 'oesophagus': ['oesophagus', 'esophagus'],
 'endometrium': ['endometrium'],
 'central_nervous_system': ['central nervous system',
  'nervous system',
  'glioblastoma',
  'neurofibromatosis',
  'anaplastic astrocytoma'],
 'soft_tissue': ['soft tissue'],
 'pancreas': ['pancreas', 'pancreatic'],
 'fibroblast': ['fibroblast'],
 'stomach': ['stomach', 'abdomen', 'gastric', 'gastrointestinal'],
 'kidney': [

In [11]:
#add spaces around each term so we don't have subset of words like 
print("os" in "bospo")

for k, v in search_terms.items():
    search_terms[k] = list(map(lambda x: " "+x+" ", v))
    search_terms[k].extend(list(map(lambda x: "-"+x+" ", v)))
    search_terms[k].extend(list(map(lambda x: " "+x+"-", v)))
    search_terms[k].extend(list(map(lambda x: x+"-", v)))
    search_terms[k].extend(list(map(lambda x: x+",", v)))
    search_terms[k].extend(list(map(lambda x: x+".", v)))
    search_terms[k].extend(list(map(lambda x: "-" + x, v)))
    search_terms[k].extend(list(map(lambda x: "/" + x, v)))
    search_terms[k].extend(list(map(lambda x: x+"/", v)))
    
search_terms


True


{'lung': [' lung ',
  ' pulmonary ',
  ' lungs ',
  '-lung ',
  '-pulmonary ',
  '-lungs ',
  ' lung-',
  ' pulmonary-',
  ' lungs-',
  'lung-',
  'pulmonary-',
  'lungs-',
  'lung,',
  'pulmonary,',
  'lungs,',
  'lung.',
  'pulmonary.',
  'lungs.',
  '-lung',
  '-pulmonary',
  '-lungs',
  '/lung',
  '/pulmonary',
  '/lungs',
  'lung/',
  'pulmonary/',
  'lungs/'],
 'large_intestine': [' large intestine ',
  '-large intestine ',
  ' large intestine-',
  'large intestine-',
  'large intestine,',
  'large intestine.',
  '-large intestine',
  '/large intestine',
  'large intestine/'],
 'haematopoietic_and_lymphoid_tissue': [' haematopoietic and lymphoid tissue ',
  ' haematopoietic ',
  ' lymphoid ',
  '-haematopoietic and lymphoid tissue ',
  '-haematopoietic ',
  '-lymphoid ',
  ' haematopoietic and lymphoid tissue-',
  ' haematopoietic-',
  ' lymphoid-',
  'haematopoietic and lymphoid tissue-',
  'haematopoietic-',
  'lymphoid-',
  'haematopoietic and lymphoid tissue,',
  'haematopoie

Now loop and generate tags

In [12]:
tags = {data.iloc[i]["NAME"]:[] for i in range(data.shape[0])}

unassigned = []

for item in nonempty:
    assigned = False
    cancer_tags = []
    index = data["NAME"] == item
    
    summary = data[index]["Summary"].to_numpy()[0]
    background = data[index]["Background"].to_numpy()[0]
    indication = data[index]["Indication"].to_numpy()[0]
    
    for ctype, terms in search_terms.items():
        for term in terms:
            if type(summary)!=float:
                if term in summary.lower():
                    cancer_tags.append(ctype.upper())
                    assigned = True

            if type(background)!=float:
                if term in background.lower():
                    cancer_tags.append(ctype.upper())
                    assigned = True

            if type(indication)!=float:
                if term in indication.lower():
                    cancer_tags.append(ctype.upper())
                    assigned = True

    tags[item] = list(set(cancer_tags))
    if not assigned:
        unassigned.append(item)
    
tags

{'Afatinib 1/2': ['LUNG'],
 'Afatinib 2/2': ['LUNG'],
 'AKT inhibitor VIII 1/2': [],
 'AKT inhibitor VIII 2/2': [],
 'Avagacestat 1/2': [],
 'Avagacestat 2/2': [],
 'AZD6482 1/2': [],
 'AZD6482 2/2': [],
 'Bicalutamide 1/2': ['PROSTATE'],
 'Bicalutamide 2/2': ['PROSTATE'],
 'BMS-536924 1/2': [],
 'BMS-536924 2/2': [],
 'CHIR-99021 1/2': [],
 'CHIR-99021 2/2': [],
 'GSK269962A 1/2': [],
 'GSK269962A 2/2': [],
 'JQ1 1/2': [],
 'JQ1 2/2': [],
 'Olaparib 1/2': ['PANCREAS', 'OVARY', 'BREAST', 'PROSTATE'],
 'Olaparib 2/2': ['PANCREAS', 'OVARY', 'BREAST', 'PROSTATE'],
 'Pictilisib 1/2': ['BREAST'],
 'Pictilisib 2/2': ['BREAST'],
 'PLX-4720 1/2': [],
 'PLX-4720 2/2': [],
 'Refametinib 1/2': [],
 'Refametinib 2/2': [],
 'Selumetinib 1/2': ['CENTRAL_NERVOUS_SYSTEM'],
 'Selumetinib 2/2': ['CENTRAL_NERVOUS_SYSTEM'],
 'UNC0638 1/2': [],
 'UNC0638 2/2': [],
 '(5Z)-7-Oxozeaenol': [],
 '5-Fluorouracil': ['PANCREAS',
  'STOMACH',
  'BILIARY_TRACT',
  'BREAST',
  'KIDNEY'],
 'A-443654': [],
 'A-770041':

In [13]:
unassigned

['Avagacestat 1/2',
 'Avagacestat 2/2',
 'AZD6482 1/2',
 'AZD6482 2/2',
 'PLX-4720 1/2',
 'PLX-4720 2/2',
 'Refametinib 1/2',
 'Refametinib 2/2',
 'AICA Ribonucleotide',
 'AR-42',
 'AZD7762',
 'AZD8055',
 'Belinostat',
 'Bleomycin',
 'BMS-754807',
 'CCT-018159',
 'Doramapimod',
 'Fedratinib',
 'GSK690693',
 'Linsitinib',
 'Masitinib',
 'Motesanib',
 'OSI-027',
 'Parthenolide',
 'PHA-793887',
 'Phenformin',
 'Pyrimethamine',
 'Selisistat',
 'Serdemetan',
 'TAK-715',
 'Veliparib',
 'VX-702']

In [14]:
data[data["NAME"] == "Rapamycin"]

,NAME,Summary,DrugBank Accession Number,Background,Groups,Weight,Indication
192,Rapamycin,"Sirolimus is an mTOR inhibitor immunosuppressant used to prevent organ transplant rejections, treat lymphangioleiomyomatosis, and treat adults with perivascular epithelioid cell tumors.",DB00877,"Sirolimus, also known as rapamycin, is a macrocyclic lactone antibiotic produced by bacteria Streptomyces hygroscopicus, which was isolated from the soil of the Vai Atari region of Rapa Nui (Easter Island).5 It was first isolated and identified as an antifungal agent with potent anticandida activity; however, after its potent antitumor and immunosuppressive activities were later discovered, it was extensively investigated as an immunosuppressive and antitumour agent.3 Its primary mechanism of action is the inhibition of the mammalian target of rapamycin (mTOR), which is a serine/threonine-specific protein kinase that regulates cell growth, proliferation, and survival. mTOR is an important therapeutic target for various diseases, as it was shown to regulate longevity and maintain normal glucose homeostasis.6 Targeting mTOR received more attention especially in cancer, as mTOR signalling pathways are constitutively activated in many types of human cancer.1\nSirolimus was first approv...","Approved, Investigational",914.187,"Sirolimus is indicated for the prophylaxis of organ rejection in patients aged 13 years or older receiving renal transplants. In patients at low-to moderate-immunologic risk, it is recommended that sirolimus be used initially in a regimen with cyclosporine and corticosteroids; cyclosporine should be withdrawn two to four months after transplantation. In patients at high-immunologic risk (defined as Black recipients and/or repeat renal transplant recipients who lost a previous allograft for immunologic reason and/or patients with high panel-reactive antibodies [PRA; peak PRA level > 80%]), it is recommended that sirolimus be used in combination with cyclosporine and corticosteroids for the first year following transplantation.8\nIt is also used to treat lymphangioleiomyomatosis.8 \nIn the US, albumin-bound sirolimus for intravenous injection is indicated for the treatment of adult patients with locally advanced unresectable or metastatic malignant perivascular epithelioid cell tumou..."


In [17]:
#add our results to the dataframe 
labels = []
for v in tags.values():
    if len(v)==1:
        labels.append(v[0])
    elif len(v)>1:
        labels.append(', '.join(v))
    else:
        labels.append("")
        
data["CancerType"] = labels

In [23]:
print(f'fraction of drugs which we found info on: {len(nonempty)/data.shape[0]}') 
print(f'fraction of drugs w/ info we could not label: {len(unassigned)/len(nonempty)}')
print(f'fraction of total drugs labelled: {(len(nonempty)/data.shape[0])*(1-len(unassigned)/len(nonempty))}')

fraction of drugs which we found info on: 0.4398496240601504
fraction of drugs w/ info we could not label: 0.27350427350427353
fraction of total drugs labelled: 0.31954887218045114


In [19]:
not_approved = data[data["Groups"]!="Approved"]
no_label_not_approved = not_approved[not_approved["CancerType"]==""]
no_label = data[data["CancerType"]==""]

print(f'fraction of not explicitly approved: {not_approved.shape[0]/data.shape[0]}')
print(f'proportion of unapproved drugs unlabelled to total unlabelled: {no_label_not_approved.shape[0]/no_label.shape[0]}')


fraction of not explicitly approved: 0.9135338345864662
proportion of unapproved drugs unlabelled to total unlabelled: 1.0


In [30]:
#unapproved drugs account for 73% of total labelled drugs 
not_approved[not_approved["CancerType"]!=""].shape[0]/data[data["CancerType"]!=""].shape[0]

0.7294117647058823

In [21]:
labels

['LUNG',
 'LUNG',
 '',
 '',
 '',
 '',
 '',
 '',
 'PROSTATE',
 'PROSTATE',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'PANCREAS, OVARY, BREAST, PROSTATE',
 'PANCREAS, OVARY, BREAST, PROSTATE',
 'BREAST',
 'BREAST',
 '',
 '',
 '',
 '',
 'CENTRAL_NERVOUS_SYSTEM',
 'CENTRAL_NERVOUS_SYSTEM',
 '',
 '',
 '',
 'PANCREAS, STOMACH, BILIARY_TRACT, BREAST, KIDNEY',
 '',
 '',
 '',
 'LUNG',
 'LUNG',
 '',
 '',
 '',
 'BONE',
 'PANCREAS, THYROID, KIDNEY',
 '',
 '',
 '',
 '',
 '',
 'LUNG, SKIN, BREAST',
 'LUNG',
 '',
 '',
 '',
 '',
 '',
 '',
 'BONE',
 'BONE',
 '',
 '',
 '',
 'THYROID, KIDNEY',
 '',
 '',
 '',
 'LUNG, SKIN',
 '',
 '',
 'LUNG, PANCREAS, BREAST',
 'LUNG, OVARY',
 '',
 '',
 '',
 'LUNG',
 'LIVER, STOMACH, BREAST',
 '',
 '',
 'BONE',
 'LUNG, THYROID, SKIN',
 '',
 'BREAST, KIDNEY',
 'SKIN, BONE',
 'BONE, HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
 '',
 'OVARY, STOMACH, PROSTATE, LUNG, BREAST',
 '',
 'OVARY, THYROID, BONE, SOFT_TISSUE, STOMACH, BREAST',
 '',
 'SKIN',
 '',
 'LUNG, BREAST',
 'LUNG, CENTRA

In [22]:
data.to_csv("drugs_labelled.csv")